### Functions include:
(1) show missing data percentage in all columns

(2) remove missing data percentage >= 70% columns

(3) sort date frame time

(4) imputation

## (1) Functions

In [1]:
import pandas as pd
import numpy as np

In [2]:
def show_col_missing_data_percentage(pd1_1):
    col_name = list(pd1_1.columns.values)
    j = 0
    for i in col_name:
        # count number of rows with missing values
        n_miss = pd1_1[[i]].isnull().sum()
        perc = n_miss / pd1_1.shape[0] * 100
        print('> %d, Missing: %d (%.1f%%) -> %s' % (
            j, n_miss, perc, i))
        j = j+1

In [3]:
def remove_missing_data_columns(pd1_1, missing_percent_threshold):
    percent_missing = pd1_1.isnull().sum() * 100 / len(pd1_1)
    missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
    remove_cols = []
    for i in range(0, missing_value_df.shape[0]):
        if missing_value_df.iloc[i, 0] >= missing_percent_threshold:
            remove_cols.append(i)
    pd1_2 = pd1_1.drop(pd1_1.columns[remove_cols], axis=1) 
    
    pd1_2.loc[pd1_2['IS_MALE'] == False, 'IS_MALE'] = "F"
    pd1_2.loc[pd1_2['IS_MALE'] == True, 'IS_MALE'] = "M"
    pd1_2.rename({'IS_MALE': 'GENDER'}, axis=1, inplace=True)
    
    pd1_cols1 = pd1_2.columns.tolist()
    pd1_cols2 = pd1_cols1[0:3]+pd1_cols1[4:]
    pd1_3 = pd1_2[pd1_cols2]
    return pd1_3

In [4]:
def sort_df_time(pd1):
    hadm_id_g = pd1.groupby('HADM_ID')
    hadm_id_g_size_count = hadm_id_g.size().reset_index(
        name='counts')
    hadm_id_arr = hadm_id_g_size_count['HADM_ID'].to_numpy() 
    append_dfs = []
    
    for i in range(0, len(hadm_id_arr)):
        g = hadm_id_g.get_group(hadm_id_arr[i])
        g2 = g.sort_values(by=['WINDOW_START'])
        append_dfs.append(g2)
    pd2 = pd.concat(append_dfs)
    return pd2

In [5]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

def impute_data(input_df):
    input_df = input_df.reset_index(drop=True)   
    pd1_1 = input_df.iloc[:, 0:6] 
    pd2_1 = input_df.iloc[:, 6:] 
    X = pd2_1.values
    imputer = IterativeImputer(sample_posterior=True)
    imputer.fit(X)
    Xtrans = imputer.transform(X)
    Xtrans_df = pd.DataFrame(Xtrans)
    col_name2 = list(pd2_1.columns.values)
    Xtrans_df.columns = col_name2
    final_df2 = pd.concat([pd1_1, Xtrans_df], axis=1)
    return final_df2

## (2) Execute functions

In [6]:
m3 = '2_mimic3_6tw_6ts.csv'
m3_1 = pd.read_csv(m3)
m3_2 = remove_missing_data_columns(m3_1, 70)
m3_3 = sort_df_time(m3_2)
m3_3

,HADM_ID,WINDOW_START,WINDOW_END,AKI_IN_ENCOUNTER,AGE,GENDER,BUN,pH,PT,MCV,...,"Calcium, Total",Anion Gap,Potassium,Lactate,"Bilirubin, Total",Creatinine,Phosphate,INR(PT),PTT,RDW
0,100001,2117-09-11 08:22:00,2117-09-11 14:22:00,True,35,F,NaN,NaN,12.1,80.0,...,NaN,NaN,4.9,1.9,0.5,2.60,NaN,1.0,27.0,13.6
1,100001,2117-09-11 14:22:00,2117-09-11 20:22:00,True,35,F,NaN,NaN,12.1,82.0,...,8.85,18.0,4.2,1.9,0.5,2.35,3.05,1.0,27.0,14.3
2,100001,2117-09-11 20:22:00,2117-09-12 02:22:00,True,35,F,NaN,NaN,12.1,82.0,...,7.00,16.0,4.1,1.9,0.5,2.30,2.50,1.0,27.0,14.3
3,100001,2117-09-12 02:22:00,2117-09-12 08:22:00,True,35,F,NaN,NaN,12.1,81.0,...,9.20,13.0,3.8,1.9,0.5,2.20,2.80,1.0,27.0,13.9
4,100001,2117-09-12 08:22:00,2117-09-12 14:22:00,True,35,F,NaN,NaN,12.1,81.0,...,9.20,13.0,3.8,1.9,0.5,2.20,2.80,1.0,27.0,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838833,181266,2198-05-06 20:07:00,2198-05-07 02:07:00,False,58,M,NaN,NaN,13.9,81.0,...,8.50,16.0,3.6,NaN,NaN,0.70,2.90,1.3,28.7,14.9
838834,181266,2198-05-07 02:07:00,2198-05-07 08:07:00,False,58,M,NaN,NaN,13.5,84.0,...,8.50,13.0,3.9,NaN,NaN,0.80,2.90,1.2,29.5,14.9
838835,181269,2126-07-25 12:10:00,2126-07-25 18:10:00,True,41,M,12.0,NaN,12.1,81.0,...,9.10,14.0,3.4,NaN,0.3,0.70,3.90,1.0,24.0,13.5
838836,181269,2126-07-25 18:10:00,2126-07-26 00:10:00,True,41,M,12.0,NaN,12.1,81.0,...,9.10,14.0,4.0,NaN,0.3,0.70,3.90,1.0,26.9,13.5


In [6]:
m3 = '2_mimic3_6tw_6ts.csv'
m3_1 = pd.read_csv(m3)
m3_2 = remove_missing_data_columns(m3_1, 70)
m3_3 = sort_df_time(m3_2)
m3_4 = impute_data(m3_3)

In [7]:
m4 = '2_mimic4_6tw_6ts.csv'
m4_1 = pd.read_csv(m4)
m4_2 = remove_missing_data_columns(m4_1, 70)
m4_3 = sort_df_time(m4_2)
m4_4 = impute_data(m4_3)

In [8]:
eicu = '2_eicu_6tw_6ts.csv'
eicu_1 = pd.read_csv(eicu)
eicu_2 = remove_missing_data_columns(eicu_1, 70)
eicu_3 = sort_df_time(eicu_2)
eicu_4 = impute_data(eicu_3)

KeyboardInterrupt: 

In [ ]:
e_c2 = eicu_4.columns.values.tolist()
m3_c2 = m3_4.columns.values.tolist()
common_ele1 = np.array(list(set(e_c2).intersection(m3_c2)))
m4_c2 = m4_4.columns.values.tolist()
common_eles = np.array(list(set(common_ele1).intersection(m4_c2)))

In [ ]:
common_eles

In [9]:
common_eles2 = ['HADM_ID', 'WINDOW_START', 'WINDOW_END', 
                'AKI_IN_ENCOUNTER', 'GENDER', 'AGE',
                'Potassium', 'MCV', 'Sodium', 'Calcium, Total', 
                'Phosphate', 'Magnesium', 'INR(PT)', 'RDW', 
                'Anion Gap', 'PT', 'pH', 'BUN', 'Lactate', 
                'Creatinine', 'Bilirubin, Total', 'PTT']

In [10]:
eicu_5 = eicu_4[common_eles2]
m3_5 = m3_4[common_eles2]
m4_5 = m4_4[common_eles2]

In [11]:
m4_5

,HADM_ID,WINDOW_START,WINDOW_END,AKI_IN_ENCOUNTER,GENDER,AGE,Potassium,MCV,Sodium,"Calcium, Total",...,INR(PT),RDW,Anion Gap,PT,pH,BUN,Lactate,Creatinine,"Bilirubin, Total",PTT
0,20000094,2150-03-02 15:34:00,2150-03-02 21:34:00,False,M,79,4.600000,86.000000,133.00000,9.300000,...,1.700000,20.500000,25.0,18.200000,7.410,99.0,4.000000,3.8,2.500000,37.400000
1,20000094,2150-03-02 21:34:00,2150-03-03 03:34:00,False,M,79,4.600000,86.000000,133.00000,9.300000,...,1.700000,20.500000,25.0,18.200000,7.360,99.0,4.500000,3.8,2.500000,37.400000
2,20000094,2150-03-03 03:34:00,2150-03-03 09:34:00,False,M,79,4.300000,89.000000,127.00000,7.800000,...,2.000000,20.400000,25.0,22.100000,7.340,99.0,6.900000,3.9,2.700000,40.200000
3,20000147,2121-08-30 16:30:00,2121-08-30 22:30:00,False,M,71,4.056098,90.333333,141.76538,7.840302,...,1.266667,12.233333,17.0,13.733333,7.375,12.0,1.475000,0.7,-3.179305,66.733333
4,20000147,2121-08-30 22:30:00,2121-08-31 04:30:00,False,M,71,4.600000,91.000000,134.00000,8.707846,...,1.266667,12.400000,14.0,13.733333,7.390,13.0,1.900000,0.7,1.942723,66.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143334,29999625,2157-11-08 00:00:00,2157-11-08 06:00:00,True,M,69,4.100000,86.000000,149.00000,8.600000,...,1.200000,14.600000,15.0,13.200000,7.410,21.0,1.300000,1.6,-2.325518,25.400000
2143335,29999828,2180-10-28 08:15:00,2180-10-28 14:15:00,False,M,47,4.700000,90.000000,135.00000,8.700000,...,1.100000,13.600000,12.0,11.800000,7.270,13.0,2.368009,0.9,2.374469,28.100000
2143336,29999828,2180-10-28 14:15:00,2180-10-28 20:15:00,False,M,47,4.700000,90.000000,135.00000,8.700000,...,1.100000,13.600000,12.0,11.800000,7.270,13.0,1.074661,0.9,1.281016,28.100000
2143337,29999828,2180-10-28 20:15:00,2180-10-29 02:15:00,False,M,47,4.700000,90.000000,135.00000,8.700000,...,1.100000,13.600000,12.0,11.800000,7.270,13.0,1.522087,0.9,-1.088781,28.100000


In [12]:
m3_5.to_csv("3_mimic3_6tw_6ts_imputed.csv", index=False)
m4_5.to_csv("3_mimic4_6tw_6ts_imputed.csv", index=False)
eicu_5.to_csv("3_eicu_6tw_6ts_imputed.csv", index=False)